In [ ]:
!python -m pip install -r requirements.txt

# Discrete Fourier transform

In [ ]:
import numpy as np

def dft_matrix(n: int, inv: bool = False) -> np.ndarray:
    if not isinstance(n, int) or not isinstance(inv, bool):
        raise TypeError()
    if n < 1:
        raise ValueError()
        
    k = np.arange(n)
    w_n = np.exp((2J if inv else -2J) * k * np.pi / n)
    return np.vander(w_n, n, increasing=True) / (n if inv else 1.0)

In [ ]:
from numpy.fft import fft, ifft
from numpy.random import normal, randint, randn

for _ in range(100):
    kwargs = {'rtol': 1E-8, 'atol': 1E-8}
    n = randint(100) + 1
    x = randn(n) + 1J * randn(n)

    y = dft_matrix(n) @ x
    assert np.allclose(y, fft(x), **kwargs)
    x = dft_matrix(n, inv=True) @ y
    assert np.allclose(x, ifft(y), **kwargs)

In [ ]:
from math import cos, pi, sin

def f(x: float) -> float:
    return 7.0 * sin(x) + 5.0 * cos(x * 7.0)

start, stop = -2.0 * pi, 2.0 * pi
x = np.linspace(start, stop, 2 ** 10 + 1)
f_vectorize = np.vectorize(f)
y = f_vectorize(x)

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(16, 9), dpi=400)
subplot = fig.add_subplot(111, facecolor='#FFFFFF')
subplot.plot(x, y, color='red', lw=2, label='y')
plt.legend()
plt.show()

In [ ]:
n = len(x)
y_noise = y + normal(loc=0.0, scale=1.0, size=n)
fig = plt.figure(figsize=(16, 9), dpi=400)
subplot = fig.add_subplot(111, facecolor='#FFFFFF')
subplot.plot(x, y_noise, 'Xr', label='y_noise')
subplot.plot(x, y, color='green', lw=2, label='y')
plt.legend()
plt.show()

In [ ]:
x_upper = dft_matrix(n)
dft = x_upper @ y_noise

fig = plt.figure(figsize=(16, 9), dpi=400)
subplot = fig.add_subplot(211, facecolor='#FFFFFF')
subplot.plot(np.abs(dft[:n // 2]), 'bo', label='dft: abs')
subplot = fig.add_subplot(212, facecolor='#FFFFFF')
subplot.plot(np.angle(dft), 'ro', label='dft: angle')
plt.legend()
plt.show()

In [ ]:
epsilon = 100.0
dft_filter = np.where(np.abs(dft) < epsilon, 0.0, dft)
x_upper_inv = dft_matrix(n, inv=True)
y_filter = x_upper_inv @ dft_filter
assert np.all(np.imag(y_filter) <= 1.0E-8)
y_filter = np.real(y_filter)

fig = plt.figure(figsize=(16, 9), dpi=400)
subplot = fig.add_subplot(111, facecolor='#FFFFFF')
subplot.plot(x, y_noise, 'Xr', lw=2, label='y_noise')
subplot.plot(x, y, color='green', lw=2, label='y')
subplot.plot(x, y_filter, color='blue', label='y_filter')
plt.legend()
plt.show()

# Hybrid basic 

In [ ]:
def dft_lsq(x: np.ndarray, y: np.ndarray, order: int = 3,
            epsilon: float = 1E-7) -> np.ndarray:
    if (not isinstance(x, np.ndarray) or not isinstance(y, np.ndarray)
            or not isinstance(order, int) or not isinstance(epsilon, float)):
        raise TypeError()
    n, k = len(x), order + 1
    if (len(x.shape) != 1 or n < 2 or np.any(np.diff(x) <= 0)
            or x.shape != y.shape or order not in range(1, 5)
            or epsilon < float_info.epsilon):
        raise ValueError()
    
    t_upper = x[-1] - x[0] # ???
    x_upper = np.zeros(shape=(n, k + m), dtype=complex)
    x_upper[:, :k] = dft_matrix(n)
    for j in range(m):
        index = x > knots[j]
        x_upper[index, k + j] += np.power(x[index] - knots[j], order)
    left, right = x_upper.T @ x_upper, x_upper.T @ y
    beta = solve(left, right)
    if not robust:
        return x_upper @ beta

    for _ in range(max_iter):
        x_upper_beta = x_upper @ beta
        diff = x_upper_beta - y
        cond = np.abs(diff) < delta
        y = np.where(cond, y, x_upper_beta - np.sign(diff) * delta)
        right = x_upper.T @ y
        next_beta = solve(left, right)
        error = norm(next_beta - beta, ord=2)
        beta, next_beta = next_beta, beta
        if (error < epsilon):
            break
    return x_upper @ beta